In [ ]:
# In[1]:

import pandas as pd

# Read metric_container.csv and filter by allowed components
df_container = pd.read_csv('dataset/Bank/telemetry/2021_03_04/metric/metric_container.csv')

allowed_components = [
    'apache01', 'apache02',
    'Tomcat01', 'Tomcat02', 'Tomcat03', 'Tomcat04',
    'MG01', 'MG02',
    'IG01', 'IG02',
    'Mysql01', 'Mysql02',
    'Redis01', 'Redis02'
]

df_filtered = df_container[df_container['cmdb_id'].isin(allowed_components)].copy()

# Calculate P5 and P95 thresholds for each component-KPI pair
thresholds_df = df_filtered.groupby(['cmdb_id', 'kpi_name'])['value'].agg(
    p5=lambda x: x.quantile(0.05),
    p95=lambda x: x.quantile(0.95)
).reset_index()

# Display the result
thresholds_df

```
Out[1]:
```


The analysis calculated 95th percentile (P95) and 5th percentile (P5) thresholds for 1,643 component-KPI pairs across all specified possible root cause components (e.g., Tomcat, Mysql, Redis, etc.). These thresholds represent the expected operating range of system metrics like CPU load, memory usage, and disk I/O for each component. For example:  

- **IG01** has a P95 heap memory usage of 48.76 MB (vs. P5 at 10.68 MB), indicating typical memory consumption patterns.  
- **Apache02** shows zero swap memory usage (Si/So metrics), suggesting no active swapping occurred during the day.  

These thresholds can serve as baselines to detect abnormalities. For instance, if a component's real-time metric exceeds its P95 value, it may indicate resource contention or performance degradation. The full dataset covers critical infrastructure components, ensuring comprehensive coverage of potential root causes.

The original code execution output of IPython Kernel is also provided below for reference:

cmdb_id                                          kpi_name            p5           p95
0         IG01          JVM-Memory_7778_JVM_Memory_HeapMemoryMax  2.040136e+09  2.040136e+09
1         IG01        JVM-Memory_7778_JVM_Memory_HeapMemoryUsage  1.068285e+01  4.875715e+01
2         IG01         JVM-Memory_7778_JVM_Memory_HeapMemoryUsed  2.180178e+08  9.901924e+08
3         IG01       JVM-Memory_7778_JVM_Memory_NoHeapMemoryUsed  8.265500e+08  9.432373e+08
4         IG01         JVM-Operating System_7778_JVM_JVM_CPULoad  5.480000e-02  3.350250e-01
...        ...                                               ...           ...           ...
1638  apache02                      OSLinux-OSLinux_SWAP_SWAP_Si  0.000000e+00  0.000000e+00
1639  apache02                      OSLinux-OSLinux_SWAP_SWAP_So  0.000000e+00  0.000000e+00
1640  apache02  OSLinux-OSLinux_SYSTEM_SYSTEM_Check-DefaultRoute  1.000000e+00  1.000000e+00
1641  apache02      OSLinux-OSLinux_SYSTEM_SYSTEM_Check-Hostname  1.000000e+00  1.000000e+00
1642  apache02                OSLinux-OSLinux_ZABBIX_Host_Uptime  5.989793e+06  6.067511e+06

[1643 rows x 4 columns]

**Note**: The printed pandas DataFrame is truncated due to its size. Only **10 rows** are displayed, which may introduce observation bias due to the incomplete table. If you want to comprehensively understand the details without bias, please ask Executor using `df.head(X)` to display more rows.```
```

In [ ]:
# In[2]:

import pandas as pd
from datetime import datetime
import pytz

# Convert failure window times to epoch seconds in UTC+8
tz = pytz.timezone('Asia/Shanghai')
start_dt = tz.localize(datetime(2021, 3, 4, 18, 0, 0))
end_dt = tz.localize(datetime(2021, 3, 4, 18, 30, 0))
start_epoch = int(start_dt.timestamp())
end_epoch = int(end_dt.timestamp())

# Read and filter metric_container data for the failure window and allowed components
allowed_components = [
    'apache01', 'apache02', 'Tomcat01', 'Tomcat02', 'Tomcat03', 'Tomcat04',
    'MG01', 'MG02', 'IG01', 'IG02', 'Mysql01', 'Mysql02', 'Redis01', 'Redis02'
]
filtered_df = pd.read_csv('dataset/Bank/telemetry/2021_03_04/metric/metric_container.csv')
filtered_df = filtered_df[filtered_df['cmdb_id'].isin(allowed_components)]
filtered_df = filtered_df[(filtered_df['timestamp'] >= start_epoch) & (filtered_df['timestamp'] <= end_epoch)]

# Merge with pre-calculated thresholds and identify anomalies
merged_df = pd.merge(filtered_df, thresholds_df, on=['cmdb_id', 'kpi_name'], how='left')

def calculate_breach(row):
    value = row['value']
    p5 = row['p5']
    p95 = row['p95']
    
    if value < p5:
        if p5 == 0:
            return float('nan')  # Skip if p5=0 (non-negative KPIs)
        return (p5 - value) / p5 * 100
    elif value > p95:
        if p95 == 0:
            return 100.0  # Mark zero-based KPIs exceeding baseline as 100% breach
        return (value - p95) / p95 * 100
    else:
        return float('nan')

merged_df['breach_percentage'] = merged_df.apply(calculate_breach, axis=1)
anomalies_df = merged_df[merged_df['breach_percentage'].notnull()][['cmdb_id', 'kpi_name', 'timestamp', 'value', 'breach_percentage']]

# Display the anomalies
anomalies_df.sort_values('breach_percentage', ascending=False).head(20)

```
Out[2]:
```


The analysis detected anomalies in system metrics during the 30-minute failure window (18:00–18:30 UTC+8) across multiple components. Key findings include:

1. **Mysql02 Disk I/O Spike**  
   - **34,800% breach** in disk read throughput (`LOCALDISK-sdb_DSKRead`) far exceeding P95 thresholds, suggesting potential disk saturation or query overload.

2. **Mysql01 Lock Contention**  
   - **17,733% breach** in InnoDB row lock time (`Innodb Row Lock Time`), indicating severe resource contention or inefficient transactions.

3. **Persistent Disk I/O and CPU Wait Issues**  
   - Multiple components (Mysql, Redis, IG, Apache, Tomcat) showed **breach percentages exceeding 1000%** for disk-related KPIs (e.g., `DSKRead`, `DSKRTps`) and CPU wait I/O (`CPUWio`), signaling systemic resource contention.

4. **Root Cause Hypotheses**  
   - Mysql's disk I/O saturation likely cascaded into lock contention (Mysql01) and triggered CPU bottlenecks in dependent services (IG, Redis, Tomcat).  
   - High CPU wait I/O across components suggests a possible shared storage bottleneck or network-attached storage latency.

**Immediate action**: Investigate Mysql02's disk subsystem (sdb/sdc) performance and query patterns during this window. Validate if thresholds were calculated correctly (e.g., avoid zero-value baselines) and correlate with trace/log data for latency spikes.

The original code execution output of IPython Kernel is also provided below for reference:

cmdb_id                                         kpi_name   timestamp       value  breach_percentage
699     Mysql02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sdb_DSKRead  1614852000   46.533333       34800.087000
778     Mysql01            Mysql-MySQL_3306_Innodb Row Lock Time  1614852060   35.666700       17733.350000
7008       IG02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sda_DSKRead  1614852480  329.200000        5511.370013
704     Mysql02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sdc_DSKRead  1614852000    7.333333        5400.013500
703     Mysql02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sdc_DSKRTps  1614852000    0.916667        5389.023952
9922    Redis02                           OSLinux-CPU_CPU_CPUWio  1614852660    3.565775        5079.048656
17775      IG02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sda_DSKRTps  1614853200    6.050000        4437.511344
691     Mysql02   OSLinux-OSLinux_LOCALDISK_LOCALDISK-sda_DSKBps  1614852000  179.000000        4375.000000
13430      MG01  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sda_DSKRead  1614852900  266.800000        4346.666667
9244    Redis02                           OSLinux-CPU_CPU_CPUWio  1614852600    2.417200        3410.820625
157    Tomcat04                           OSLinux-CPU_CPU_CPUWio  1614852000    3.758100        2021.781843
1215   Tomcat03                           OSLinux-CPU_CPU_CPUWio  1614852060    4.969000        1996.138027
1042   Tomcat02                           OSLinux-CPU_CPU_CPUWio  1614852060    3.147700        1549.348948
734    Tomcat01                           OSLinux-CPU_CPU_CPUWio  1614852060    3.441500        1479.394218
7009       IG02  OSLinux-OSLinux_LOCALDISK_LOCALDISK-sda_DSKRTps  1614852480    2.050000        1437.503844
13213   Redis02                           OSLinux-CPU_CPU_CPUWio  1614852900    1.058200        1436.964415
1180   apache02                           OSLinux-CPU_CPU_CPUWio  1614852060    1.711817        1347.686583
922     Redis01                           OSLinux-CPU_CPU_CPUWio  1614852060    0.819600        1263.702162
362        IG01                           OSLinux-CPU_CPU_CPUWio  1614852000    2.386400        1241.276978
1201       IG02                           OSLinux-CPU_CPU_CPUWio  1614852060    3.204500        1033.955756```
```

In [ ]:
# In[3]:

import pandas as pd

# Read and filter trace data for the failure window
trace_df = pd.read_csv('dataset/Bank/telemetry/2021_03_04/trace/trace_span.csv')
filtered_trace = trace_df[
    (trace_df['timestamp'] >= 1614852000) & (trace_df['timestamp'] <= 1614853800)
]

# Extract high-latency spans (duration >1500 ms)
high_lat_spans = filtered_trace[filtered_trace['duration'] > 1500]

# Map span_id to rows for quick lookup
span_to_row = {row['span_id']: row for _, row in filtered_trace.iterrows()}

# Correlate with Mysql01/02 in ancestry chains of high-latency spans
faulty_services = []
for _, span_row in high_lat_spans.iterrows():
    current_span = span_row['span_id']
    parent_id = span_row['parent_id']
    chain = [current_span]
    
    # Build ancestry chain (span to root within filtered trace)
    while parent_id in span_to_row:
        chain.append(parent_id)
        parent_id = span_to_row[parent_id]['parent_id']

    # Find the deepest Mysql01/02 component in chain
    for span in chain:
        cmdb_id = span_to_row[span]['cmdb_id']
        if cmdb_id in ['Mysql01', 'Mysql02']:
            faulty_services.append({
                'trace_id': span_row['trace_id'],
                'faulty_component': cmdb_id
            })
            break  # Break after first match (deepest in chain)

# Create DataFrame with explicit column definitions to avoid KeyError
trace_faulty_services = pd.DataFrame(faulty_services, columns=['trace_id', 'faulty_component'])
trace_faulty_services.drop_duplicates(inplace=True)
trace_faulty_services.sort_values('trace_id', inplace=True)

# Display results
trace_faulty_services.head(20)

```
Out[3]:
```


The trace analysis found **no evidence** of Mysql01 or Mysql02 being part of high-latency call chains during the failure window. This suggests:  

1. **Metrics vs. Traces Disconnect**:  
   - Earlier metric anomalies (Mysql disk I/O, locks) did not directly correlate with trace-level latency in this dataset. Possible reasons:  
     - Metrics captured database strain, but traces may have missed long-running queries or cross-component transactions.  
     - Traces might not instrument database calls (e.g., only HTTP/API calls are traced, not database interactions).  

2. **Alternative Root Causes**:  
   - High-latency traces (not shown here) might originate from other components with anomalies (e.g., Redis CPU wait I/O at 5,079% breach or IG disk reads at 5,511% breach).  

**Recommendations**:  
- Cross-check with **log data** (e.g., slow query logs for MySQL, JVM thread dumps for Tomcat).  
- Validate if database calls are **properly instrumented in traces** (e.g., check if span `cmdb_id` includes database connections).  
- Refine trace analysis to **include all anomaly-causing components** (e.g., Redis, IG) and not just Mysql01/02.

The original code execution output of IPython Kernel is also provided below for reference:

Empty DataFrame
Columns: [trace_id, faulty_component]
Index: []```
```

In [ ]:
# In[4]:

import pandas as pd
import pytz
from datetime import datetime

# Read and filter log_service.csv for target components and time window
target_components = ['Mysql01', 'Mysql02', 'Redis02', 'IG02']
log_df = pd.read_csv('dataset/Bank/telemetry/2021_03_04/log/log_service.csv')
filtered_log = log_df[
    (log_df['cmdb_id'].isin(target_components)) &
    (log_df['timestamp'] >= 1614852000) & 
    (log_df['timestamp'] <= 1614853800)
]

# Define regex pattern for keywords
keywords = ['disk', 'I/O', 'lock', 'wait', 'timeout', 'swap']
pattern = '|'.join([f'(?i){kw}' for kw in keywords])  # Case-insensitive matches

# Filter rows with keyword matches in 'value' column
actionable_logs = filtered_log[filtered_log['value'].str.contains(pattern, na=False)]

# Convert epoch to UTC+8 timestamp
def epoch_to_utc8(epoch):
    return datetime.utcfromtimestamp(epoch).replace(tzinfo=pytz.utc).astimezone(pytz.timezone('Asia/Shanghai'))

actionable_logs['human_time'] = actionable_logs['timestamp'].apply(epoch_to_utc8)

# Return structured output with cmdb_id, timestamp, and value
result = actionable_logs[['cmdb_id', 'human_time', 'value']].sort_values(['cmdb_id', 'human_time'])
result.head(20)

```
Out[4]:
```


The log analysis found **no actionable entries** matching keywords like disk, I/O, lock, or timeout in **MySQL, Redis02, or IG02 logs** during the failure window. This lack of evidence suggests:

1. **Potential Blind Spots**:  
   - Critical events (e.g., MySQL disk I/O saturation, lock contention) may **not be logged** at the app/service log level or use different terminology (e.g., "disk latency" instead of "I/O").  
   - Logs might require deeper parsing of structured fields (e.g., query execution time in MySQL slow logs) not included in this analysis.  

2. **Next Steps**:  
   - Cross-check with **MySQL-specific diagnostics**:  
     - Analyze metric_container.csv for disk queue depths (e.g., `OSLinux-OSLinux_LOCALDISK_LOCALDISK-sdX_DSKRun`) to confirm disk saturation.  
     - Correlate high-latency traces (even if not linked to MySQL) with application logs (e.g., Tomcat/IG) for retry patterns or timeouts.  
   - Validate log data completeness: Ensure logs from MySQL and Redis are being collected (e.g., check log file locations, rotation policies).  

The **metric data** ([previous step](#step-5)) strongly indicates disk I/O and lock issues in MySQL, even lacking direct log evidence. Prioritize metric-driven root cause analysis until logs are validated.

The original code execution output of IPython Kernel is also provided below for reference:

Empty DataFrame
Columns: [cmdb_id, human_time, value]
Index: []```
```